In [39]:
import requests
from bs4 import BeautifulSoup as soup
from bs4 import element
import re
import lxml

## Getting links and metadata from https://www.n-tv.de/news.xml

In [2]:
def extract_meta_data():
    news = soup(requests.get('https://www.n-tv.de/news.xml').text, features="xml")
    
    keywords = ['Ukraine','Russland', 'Wladimir Putin']  # ToDo Keywords spezifizieren
    
    filtered_news = [url for url in news.find('urlset').find_all('url', recursive=False) if len(set(keywords).intersection(set(url.find('news:keywords').text.split(', '))))!=0]
    
    return_list = []
    for new in filtered_news:
        dictionary = {
        'url' : new.find('loc').text,
        'title' : new.find('news:title').text,
        'date' : new.find('news:publication_date').text,
        'keywords' : new.find('news:keywords').text
        }
        dictionary = { **dictionary, 'figcaption' : new.find('image:caption').text } if new.find('image:caption') else dictionary
        return_list += [dictionary]
        
    return return_list


## Getting text and ... from urls

In [131]:
url = extract_meta_data()[5]['url']
url

'https://www.n-tv.de/panorama/Hunderte-ukrainische-Lehrer-bewerben-sich-in-Deutschland-article23286199.html'

In [43]:
type(list(article_soup.find('span', class_='article__kicker').children)[0])
isinstance(list(article_soup.find('span', class_='article__kicker').children)[0],element.NavigableString)

True

In [41]:
list(article_soup.find('span', class_='article__kicker').children)[0].name

In [127]:
def get_article_content(url):
    article_soup = soup(requests.get(url).text)
    kicker = article_soup.find('span', class_='article__kicker').text
    title =  article_soup.find('span', class_='article__headline').text
    author = article_soup.find('span', class_='article__author').text if article_soup.find('span', class_='article__author') else ''
    date = article_soup.find('span', class_='article__date').text
    strong = article_soup.find('strong').text if article_soup.find('strong') else ''
    article_text = article_soup.find('div', class_='article__text')
    text = strong + ' '.join([x for p in article_text.find_all(lambda x: x.name =='p') for x in [i for i in p.contents if isinstance(i,element.NavigableString) if i]])
    subheadline = '. '.join([h2.text for h2 in article_text.find_all('h2')])
    
    return_dict = {
        'title': title,
        'kicker': kicker,
        'author':author,
        'date':date,
        'text':text,
        'subheadlines': subheadline
    }
    
    return return_dict


In [130]:
get_article_content(extract_meta_data()[5]['url'])

{'title': 'Hunderte ukrainische Lehrer bewerben sich in Deutschland',
 'kicker': '\n                    "Willkommenskräfte" schon aktiv',
 'author': '',
 'date': '24.04.2022, 10:35 Uhr',
 'text': 'Unter den ukrainischen Geflüchteten in Deutschland sind viele Kinder. Die müssen hierzulande unterrichtet werden. Da passt es gut, dass sich mittlerweile Hunderte ukrainische Lehrer an deutschen Schulen bewerben. Allerdings könnten auch andere Branchen von weiteren Fachkräften profitieren. Die Kultusministerien der Bundesländer haben Hunderte Bewerbungen ukrainischer Lehrkräfte erhalten. In Hessen hätten sich 400 Lehrkräfte aus der Ukraine beworben, in Berlin 300, berichtete die "Welt am Sonntag" (Wams) unter Verweis auf eine Umfrage bei den Behörden. In weiteren Bundesländern sehe es ähnlich aus. Demnach haben zahlreiche Geflüchtete auch schon ihren Dienst an deutschen Schulen angetreten. Bayern hat dem "Wams"-Bericht zufolge 200 "Willkommenskräfte" mit ukrainischen Sprachkenntnissen eingest